In [112]:
import pickle
import pandas as pd
import numpy as np
import scikitplot
import scikitplot.plotters as skplt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.svm import SVC

In [2]:
with open("SavedDict.txt", "rb") as myFile:
    pulled_dic = pickle.load(myFile)

In [3]:
def trans(str0):
    if str0 == False:
        return 0
    else:
        return 1
df = pd.read_csv('zillow_feature.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df['owner_occupied'] = df['owner_occupied'].apply(trans)
df = df.drop('zestimate_percentile', axis=1)
df2 = df[['address','num_units','num_bedrooms','longitude','latitude','owner_occupied']]
cur_addr = set(df['address'])

In [4]:
rr = []
for row in df.itertuples():
    if row.address in pulled_dic:
        rr.append(row)
dff = pd.DataFrame(rr)
dff['pop'] = dff['address'].map(pulled_dic)
dff['LUC'] = dff['pop'].str[0]
dff['TotalAcres'] = dff['pop'].str[1]
dff = dff.drop(columns=['pop'])
dff = pd.get_dummies(data=dff, columns=['LUC'])
dff['TotalAcres'] = dff['TotalAcres'].astype(float).fillna(0.0)


In [5]:
# train_x2 = df2.drop('owner_occupied', axis=1).drop('address', axis=1)
# train_x2 = train_x2.apply(lambda x: (x - np.mean(x)) / (np.std(x)))

In [6]:
train_x = dff.drop('owner_occupied', axis=1)
train_y = dff['owner_occupied'].values
train_x = train_x.drop('address', axis=1)
train_x = train_x.apply(lambda x: (x - np.mean(x)) / (np.std(x)))
train_y.shape

(5813,)

In [23]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTEENN
#oversampler = SMOTEENN(random_state=0)
#oversampler=SMOTE()
oversampler=RandomOverSampler()
os_x,os_y=oversampler.fit_sample(X_train,Y_train)
#clf = XGBClassifier().fit(os_x,os_y)
#clf = XGBClassifier().fit(X_train,Y_train)
#clf = RandomForestClassifier(n_estimators=1500,max_depth=16,max_features=0.8).fit(os_x,os_y)
#clf = LogisticRegression().fit(os_x,os_y)
#clf = SVC(kernel='linear', class_weight='balanced', C=1.0, random_state=0).fit(os_x,os_y)
clf = SVC(kernel='rbf',C=10,gamma = 2).fit(os_x,os_y)
#clf = SVC().fit(os_x,os_y)
clf.score(X_test, Y_test)
#evaluate_features(os_x,os_y)
#evaluate_features(os_x,os_y,RandomForestClassifier(class_weight='balanced',n_estimators=2000, max_depth=16, verbose=1))

0.7178899082568807

In [76]:
# from sklearn.model_selection import GridSearchCV
# grid = GridSearchCV(SVC(), param_grid={'kernel': ['linear','rbf'],"C":[10, 15, 20], "gamma": [2, 1, 10, 5]}, cv=4)
# grid.fit(os_x,os_y)
# print("The best parameters are %s with a score of %0.2f"
#       % (grid.best_params_, grid.best_score_))

In [114]:
param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=300,
                                  min_samples_leaf=20,max_depth=8,max_features='sqrt', subsample=0.8,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',iid=False,cv=5)
gsearch1.fit(os_x,os_y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/Users/jxtang/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.70535, std: 0.01983, params: {'n_estimators': 20},
  mean: 0.73013, std: 0.02205, params: {'n_estimators': 30},
  mean: 0.74444, std: 0.02361, params: {'n_estimators': 40},
  mean: 0.75437, std: 0.02478, params: {'n_estimators': 50},
  mean: 0.76591, std: 0.02604, params: {'n_estimators': 60},
  mean: 0.77540, std: 0.02844, params: {'n_estimators': 70},
  mean: 0.78621, std: 0.02654, params: {'n_estimators': 80}],
 {'n_estimators': 80},
 0.7862136660592333)

In [119]:
param_test2 = {'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=80, min_samples_leaf=20, 
      max_features='sqrt', subsample=0.8, random_state=10), 
   param_grid = param_test2, scoring='roc_auc',iid=False, cv=5)
gsearch2.fit(os_x,os_y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

/Users/jxtang/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.68575, std: 0.01563, params: {'max_depth': 3, 'min_samples_split': 100},
  mean: 0.67754, std: 0.01478, params: {'max_depth': 3, 'min_samples_split': 300},
  mean: 0.66753, std: 0.01693, params: {'max_depth': 3, 'min_samples_split': 500},
  mean: 0.66622, std: 0.01696, params: {'max_depth': 3, 'min_samples_split': 700},
  mean: 0.77295, std: 0.02200, params: {'max_depth': 5, 'min_samples_split': 100},
  mean: 0.73217, std: 0.01849, params: {'max_depth': 5, 'min_samples_split': 300},
  mean: 0.71633, std: 0.01999, params: {'max_depth': 5, 'min_samples_split': 500},
  mean: 0.70543, std: 0.01816, params: {'max_depth': 5, 'min_samples_split': 700},
  mean: 0.82221, std: 0.02355, params: {'max_depth': 7, 'min_samples_split': 100},
  mean: 0.77185, std: 0.02214, params: {'max_depth': 7, 'min_samples_split': 300},
  mean: 0.75179, std: 0.02146, params: {'max_depth': 7, 'min_samples_split': 500},
  mean: 0.73212, std: 0.01720, params: {'max_depth': 7, 'min_samples_split': 700},
  me

In [120]:
param_test3 = {'min_samples_split':range(800,1900,200), 'min_samples_leaf':range(60,101,10)}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=80,max_depth=13,
                                     max_features='sqrt', subsample=0.8, random_state=10), 
                       param_grid = param_test3, scoring='roc_auc',iid=False, cv=5)
gsearch3.fit(os_x,os_y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

/Users/jxtang/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.74255, std: 0.01868, params: {'min_samples_leaf': 60, 'min_samples_split': 800},
  mean: 0.72258, std: 0.02274, params: {'min_samples_leaf': 60, 'min_samples_split': 1000},
  mean: 0.70894, std: 0.02125, params: {'min_samples_leaf': 60, 'min_samples_split': 1200},
  mean: 0.69440, std: 0.01948, params: {'min_samples_leaf': 60, 'min_samples_split': 1400},
  mean: 0.68288, std: 0.01851, params: {'min_samples_leaf': 60, 'min_samples_split': 1600},
  mean: 0.67131, std: 0.02079, params: {'min_samples_leaf': 60, 'min_samples_split': 1800},
  mean: 0.73707, std: 0.02025, params: {'min_samples_leaf': 70, 'min_samples_split': 800},
  mean: 0.71550, std: 0.02247, params: {'min_samples_leaf': 70, 'min_samples_split': 1000},
  mean: 0.70507, std: 0.02046, params: {'min_samples_leaf': 70, 'min_samples_split': 1200},
  mean: 0.69169, std: 0.02040, params: {'min_samples_leaf': 70, 'min_samples_split': 1400},
  mean: 0.68232, std: 0.02048, params: {'min_samples_leaf': 70, 'min_samples_split

In [122]:
param_test4 = {'max_features':range(7,20,2)}
gsearch4 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=80,max_depth=13, min_samples_leaf =60, 
               min_samples_split =800, subsample=0.8, random_state=10), 
                       param_grid = param_test4, scoring='roc_auc',iid=False, cv=5)
gsearch4.fit(os_x,os_y)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

/Users/jxtang/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.74956, std: 0.02320, params: {'max_features': 7},
  mean: 0.75398, std: 0.02160, params: {'max_features': 9},
  mean: 0.75613, std: 0.02504, params: {'max_features': 11},
  mean: 0.75785, std: 0.01445, params: {'max_features': 13},
  mean: 0.75878, std: 0.01681, params: {'max_features': 15},
  mean: 0.76272, std: 0.01819, params: {'max_features': 17},
  mean: 0.76110, std: 0.01706, params: {'max_features': 19}],
 {'max_features': 17},
 0.762722897712027)